In [20]:
# Import necessary libraries and utility functions
import numpy as np
import pandas as pd

from Setups.DynamicStochasticUtils import (
    generate_imperfect_grid_adjacency_matrix,
    generate_poisson_arrivals,
    generate_exponential_sojourn_times,
    create_riders_and_drivers,
    create_status_dataframe
)

# Parameters
num_nodes = 10
skip_prob = 0.15
extra_edges = 0.15
length = 100
pickup_rate = 5  # Rate of rider arrivals (Poisson)
driver_rate = 5  # Rate of driver arrivals (Poisson)
sojourn_rate = 0.4

# Generate city graph
adj_matrix = generate_imperfect_grid_adjacency_matrix(num_nodes, skip_prob, extra_edges)

# Generate time series data for demand and drivers
pickup_series = generate_poisson_arrivals(pickup_rate, (length, num_nodes))
drivers_series = generate_poisson_arrivals(driver_rate, (length, num_nodes))
sojourn_times = generate_exponential_sojourn_times(sojourn_rate, length)

# Assuming random dropoffs for simplicity
dropoffs = { (t, node): [np.random.randint(0, num_nodes - 1) for _ in range(pickup_series[t, node])] for t in range(length) for node in range(num_nodes)}

# Create riders and drivers
riders, drivers = create_riders_and_drivers(pickup_series, dropoffs, drivers_series, sojourn_times)

# Create status dataframe
status_df = create_status_dataframe(riders, drivers)
print(status_df.head())


   Time   Type Location  Patience  Sojourn Time
0     0  Rider   (0, 0)         6      5.707915
1     0  Rider   (0, 0)         1      5.707915
2     0  Rider   (0, 0)         8      5.707915
3     0  Rider   (0, 0)         7      5.707915
4     0  Rider   (0, 0)         2      5.707915


In [21]:
# Greedy Matching Algorithm and Summary Statistics

def greedy_matching(riders, drivers):
    matches = {}
    for t in riders.keys():
        matches[t] = []
        for rider in riders[t]:
            for driver in drivers[t]:
                if driver.location == rider.location and driver.patience > 0:
                    matches[t].append((rider, driver))
                    drivers[t].remove(driver)
                    break
    return matches

# Execute greedy matching
matches = greedy_matching(riders, drivers)

# Calculate summary statistics
total_riders = sum(len(riders[t]) for t in riders)
total_drivers = sum(len(drivers[t]) + len(matches[t]) for t in matches)

matched_riders = sum(len(matches[t]) for t in matches)
unmatched_riders = total_riders - matched_riders

matched_drivers = matched_riders  # Each match includes one driver
unmatched_drivers = total_drivers - matched_drivers

average_wait_time_riders = np.mean([rider.sojourn_time for t in riders for rider in riders[t] if rider.patience > 0])
average_wait_time_drivers = np.mean([driver.sojourn_time for t in drivers for driver in drivers[t] if driver.patience > 0])

print("Summary Statistics:")
print(f"Total Riders: {total_riders}")
print(f"Matched Riders: {matched_riders}")
print(f"Unmatched Riders: {unmatched_riders}")
print(f"Total Drivers: {total_drivers}")
print(f"Matched Drivers: {matched_drivers}")
print(f"Unmatched Drivers: {unmatched_drivers}")
print(f"Average Wait Time for Riders: {average_wait_time_riders:.2f}")
print(f"Average Wait Time for Drivers: {average_wait_time_drivers:.2f}")


Summary Statistics:
Total Riders: 4937
Matched Riders: 3687
Unmatched Riders: 1250
Total Drivers: 4962
Matched Drivers: 3687
Unmatched Drivers: 1275
Average Wait Time for Riders: 2.46
Average Wait Time for Drivers: 2.45
